In [33]:
import pandas as pd

# Carregar o arquivo Excel
file_path = 'Data_Airports.xlsx'
aeroportos_df = pd.read_excel(file_path).iloc[:,:3]
aeroportos_df.rename(columns={'Unnamed: 0': 'Rank'}, inplace=True)
# Exibir as primeiras linhas para entender a estrutura dos dados
aeroportos_df

,Rank,Airport,Location
0,1.0,Muscat International Airport,"Muscat, Oman"
1,2.0,Recife/Guararapes–Gilberto Freyre Internationa...,"Recife, Brazil"
2,3.0,Cape Town International Airport,"Cape Town, South Africa"
3,4.0,Brasília–Presidente Juscelino Kubitschek Inter...,"Brasília, Brazil"
4,5.0,Doha Hamad International Airport,"Doha, Qatar"
...,...,...,...
95,96.0,Thessaloniki Airport,"Thessaloniki, Greece"
96,97.0,Auckland International Airport,"Auckland, New Zealand"
97,98.0,Newcastle Airport,"Newcastle, United Kingdom"
98,99.0,Cologne Bonn Airport,"Cologne-Bonn, Germany"


In [34]:
# !pip install geopy

In [35]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import numpy as np

# Inicializar o geolocalizador
geolocator = Nominatim(user_agent="alan.b.kon@gmail.com")

# Função para obter latitude e longitude
def get_location_by_address(address):
    try:
        location = geolocator.geocode(address)
        return location.latitude, location.longitude
    except:
        return np.nan, np.nan

# Limitar a taxa de requisições para evitar bloqueio pela API
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Aplicar a função para obter as coordenadas
# NOTA: Isto pode demorar um pouco dependendo do número de localizações
aeroportos_df['Coordinates'] = aeroportos_df['Location'].apply(lambda x: get_location_by_address(x))

# Separar as coordenadas em duas colunas: Latitude e Longitude
aeroportos_df[['Latitude', 'Longitude']] = pd.DataFrame(aeroportos_df['Coordinates'].tolist(), index=aeroportos_df.index)
# Exibir as primeiras linhas para verificar
aeroportos_df.dropna(inplace=True)
aeroportos_df.head()

,Rank,Airport,Location,Coordinates,Latitude,Longitude
0,1.0,Muscat International Airport,"Muscat, Oman","(23.5882019, 58.3829448)",23.588202,58.382945
1,2.0,Recife/Guararapes–Gilberto Freyre Internationa...,"Recife, Brazil","(-8.0584933, -34.8848193)",-8.058493,-34.884819
2,3.0,Cape Town International Airport,"Cape Town, South Africa","(-33.928992, 18.417396)",-33.928992,18.417396
3,4.0,Brasília–Presidente Juscelino Kubitschek Inter...,"Brasília, Brazil","(-10.3333333, -53.2)",-10.333333,-53.200000
4,5.0,Doha Hamad International Airport,"Doha, Qatar","(25.2856329, 51.5264162)",25.285633,51.526416


In [36]:
# !pip install folium

In [37]:
import folium

# Criar um mapa base
mapa = folium.Map(location=[0, 0], zoom_start=2)

# Função para determinar a cor do marcador baseada no AirHelp Score
def cor_marcador(rank):
    if rank <= 15:
        return 'green'
    elif rank <= 45:
        return 'orange'
    else:
        return 'red'

# Adicionar marcadores para cada aeroporto
for index, row in aeroportos_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['Airport']} - Rank: {row['Rank']}",
        icon=folium.Icon(color=cor_marcador(row['Rank']))
    ).add_to(mapa)

# Salvar o mapa em um arquivo HTML
mapa_path = 'aeroportos_mapa.html'
mapa.save(mapa_path)
mapa_path

'aeroportos_mapa.html'

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import numpy as np
import folium

aeroportos_df = pd.read_excel('Data_Airports.xlsx').iloc[:,:3]
aeroportos_df.rename(columns={'Unnamed: 0': 'Rank'}, inplace=True)
geolocator = Nominatim(user_agent="alan.b.kon@gmail.com")
def get_location_by_address(address):
    try:
        location = geolocator.geocode(address)
        return location.latitude, location.longitude
    except:
        return np.nan, np.nan
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
aeroportos_df['Coordinates'] = aeroportos_df['Location'].apply(lambda x: get_location_by_address(x))
aeroportos_df[['Latitude', 'Longitude']] = pd.DataFrame(aeroportos_df['Coordinates'].tolist(), index=aeroportos_df.index)
aeroportos_df.dropna(inplace=True)

mapa = folium.Map(location=[0, 0], zoom_start=2)
def cor_marcador(rank):
    if rank <= 15:
        return 'green'
    elif rank <= 45:
        return 'orange'
    else:
        return 'red'
for index, row in aeroportos_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['Airport']} - Rank: {row['Rank']}",
        icon=folium.Icon(color=cor_marcador(row['Rank']))
    ).add_to(mapa)
mapa_path = 'aeroportos_mapa.html'
mapa.save(mapa_path)